In [ ]:
from datasets import load_dataset
import pandas as pd
from huggingface_hub import login
import random
import re

login(token="<your_token>")  # 로그인 위젯 실행 (토큰 필요)


In [ ]:
dataset = load_dataset('<your_dataset>')

In [ ]:
dataset.set_format(type='pandas')

df = dataset['train'][:]

In [ ]:
PoE_Q = []
PoE_A = []

for i, j in zip(df['answer'], df['question']):
    answer_last_char = i[-1]
    
    # A, B, C, D, E 각각에 대해 처리
    if answer_last_char in ["A", "B", "C", "D", "E"]:
        # 다른 선택지가 포함되지 않은 경우만 필터링
        if not any(f"{ch}." in i for ch in ["A", "B", "C", "D", "E"] if ch != answer_last_char):
            PoE_A.append(i)
            PoE_Q.append(j)
            
df_poe = pd.DataFrame({'question': PoE_Q, 'answer': PoE_A})

In [ ]:
poe_answer = []
poe_question = []

# 문자별 제거할 패턴과 대체 텍스트를 딕셔너리로 정의
# 각 키는 제거할 문자이며, 해당 문자가 질문 내에서 갖는 패턴을 정규식으로 처리한다.
patterns = {
    "A": (r'A\.(.*?)B\.', 'A. [공백] B.'),
    "B": (r'B\.(.*?)C\.', 'B. [공백] C.'),
    "C": (r'C\.(.*?)D\.', 'C. [공백] D.'),
    "D": (r'D\.(.*?)E\.', 'D. [공백] E.'),
    "E": (r'E\.(.*)', 'E. [공백]')
}

# 정답의 마지막 문자를 기준으로 제거 가능한 문자 집합을 정의
possible_letters = {
    "A": ['B','C','D','E'],
    "B": ['A','C','D','E'],
    "C": ['A','B','D','E'],
    "D": ['A','B','C','E'],
    "E": ['A','B','C','D']
}

for q, a in zip(df_poe['question'], df_poe['answer']):
    answer_last_char = a[-1]
    if answer_last_char in possible_letters:
        letters = possible_letters[answer_last_char]

        # 랜덤으로 2개를 선택한 후 알파벳 순서대로 정렬
        random_choices = random.sample(letters, 2)
        random_choices.sort()

        modified_text = q
        for letter in random_choices:
            pattern, repl = patterns[letter]
            modified_text = re.sub(pattern, repl, modified_text)

        poe_answer.append(a)
        poe_question.append(modified_text)
    else:
        # 해당하지 않는 경우 원문 그대로
        poe_answer.append(a)
        poe_question.append(q)


In [ ]:
added_text = (
    "다음 질문 중 가장 확률이 낮은 2가지를 소거 후 다음과 같은 문제로 치환합니다. "
    "질문에 답하는 데 가장 적합한 선택지를 선택합니다. [공백] 선택지는 무시합니다. "
)

# 답변을 바로 생성한 뒤 DataFrame에 할당
final_answers = [f"{added_text}{q}{a}" for q, a in zip(poe_question, poe_answer)]

real_df_poe = pd.DataFrame({
    'question': PoE_Q,
    'answer': final_answers
})

real_df_poe.to_parquet('train-00000-of-00001.parquet', index=False)